In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1.Installation

In [ ]:
!pip install gym==0.25.2 shimmy>=0.2.1 stable-baselines3

# 2. Import Dependencies

In [ ]:
import gym
from gym import Env
from gym.spaces import Box
import numpy as np
import random
import math
import os
from stable_baselines3 import DDPG
# from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# 3. Building an Environment

In [ ]:
class myEnv(Env):
  def __init__(self):
    self.num_tenants = 3
    # self.PNs_loc = np.random.uniform(0, 2000, (20, 2))
    self.PNs_loc = [[400, 350], [800, 350], [1200, 350], [1600, 350], \
                    [400, 700], [800, 700], [1200, 700], [1600, 700], \
                    [400, 1050], [800, 1050], [1200, 1050], [1600, 1050], \
                    [400, 1400], [800, 1400], [1200, 1400], [1600, 1400], \
                    [400, 1750], [800, 1750], [1200, 1750], [1600, 1750]]
    self.PNs_resourses = 20 * [[5000000000, 5000000000]] # computing, memory
    self.VNF_resource = [[1000, 5], [600, 4], [300, 7], [900, 4], [400, 2]] # computing, memory
    self.mMTC = [[1, 2, 3], 0.001, 100] # VNFs, delay, rate
    self.eMBB = [[1, 3, 4, 5], 0.01, 200]
    self.uRLLC = [[1, 2, 3, 4, 5], 0.1, 300]
    self.tenants_slices_request = ['mMTC', 'eMBB', 'uRLLC']
    self.avg = 0.0
    self.itr = 0

    self.state = [2, 3, 1] + 12 * [0] + 12 * [0] + 3 * [0] # sum of functions of slices is 12
    self.action_space = Box(low=np.array(12 * [0] + 12 * [1]), high=np.array(12 * [19] + 12 * [10]))
    self.observation_space = Box(low=np.array(3 * [1] + 12 * [0] + 12 * [0] + 3 * [0]), \
                                 high=np.array(3 * [10] + 12 * [10 ** 10] + 12 * [100000] + 3 * [500]))
    self.episode_length = 100000

  def link_rate(self, action):
    simultaneous = 20 * [20 * [0]]
    rate = 20 * [20 * [0]]
    noise = 10.0 ** ((-174 + (10 * math.log10(10 ** 8) + 10)) / 10.0)
    rate_state = np.zeros(12)

    for index in [0, 1, 3, 4, 5, 7, 8, 9, 10]:
      PN1 = int(action[index])
      PN2 = int(action[index + 1])
      if PN1 == PN2:
        rate[PN1][PN2] = math.inf
      elif (PN1 in range(6, 13) and PN2 in range(6, 13)) or (PN1 in range(13, 20) and PN2 in range(13, 20)):
        simultaneous[PN1][PN2] += 1
      else:
        # gain = np.random.rayleigh(scale=3.0, size=1)
        gain = 1.0
        rate[PN1][PN2] = int(action[index + 12]) * math.pow(10, 6) * math.log2(1 + (40 * gain / noise))

    for PN1 in range(20):
      for PN2 in range(20):
        if simultaneous[PN1][PN2] > 0:
          rate[PN1][PN2] = math.pow(10, 6) / simultaneous[PN1][PN2]

    for index in [0, 1, 3, 4, 5, 7, 8, 9, 10]:
      if rate[int(action[index])][int(action[index + 1])] != math.inf:
        rate_state[index] = rate[int(action[index])][int(action[index + 1])]

    return rate, rate_state

  def step(self, action):
    # with open('stochastic rewards.txt', 'a') as file:
    cpu = [0, 0, 0]
    memory = [0, 0, 0]
    consumed_cpu = 20 * [0]
    consumed_memory = 20 * [0]
    delay = np.zeros(3)
    reward = 0
    traffic_req = [0, 0, 0]
    info = {}
    info['notSLA'] = 0
    info['notResource'] = 0

    for i in range(3):
      traffic_req[i] = self.state[i] + random.uniform(-1, 1)

    for f in range(3):
      cpu[0] += self.VNF_resource[self.mMTC[0][f] - 1][0] * traffic_req[0]
      memory[0] += self.VNF_resource[self.mMTC[0][f] - 1][1] * traffic_req[0]
      consumed_cpu[int(action[f])] += self.VNF_resource[self.mMTC[0][f] - 1][0] * traffic_req[0]
      consumed_memory[int(action[f])] += self.VNF_resource[self.mMTC[0][f] - 1][1] * traffic_req[0]

    for f in range(4):
      cpu[1] += self.VNF_resource[self.eMBB[0][f] - 1][0] * traffic_req[1]
      memory[1] += self.VNF_resource[self.eMBB[0][f] - 1][1] * traffic_req[1]
      consumed_cpu[int(action[f + 3])] += self.VNF_resource[self.eMBB[0][f] - 1][0] * traffic_req[1]
      consumed_memory[int(action[f + 3])] += self.VNF_resource[self.eMBB[0][f] - 1][1] * traffic_req[1]

    for f in range(5):
      cpu[2] += self.VNF_resource[self.uRLLC[0][f] - 1][0] * traffic_req[2]
      memory[2] += self.VNF_resource[self.uRLLC[0][f] - 1][1] * traffic_req[2]
      consumed_cpu[int(action[f + 7])] += self.VNF_resource[self.uRLLC[0][f] - 1][0] * traffic_req[2]
      consumed_memory[int(action[f + 7])] += self.VNF_resource[self.uRLLC[0][f] - 1][1] * traffic_req[2]

    for n in range(20):
      if (consumed_cpu[n] > self.PNs_resourses[n][0]) or (consumed_memory[n] > self.PNs_resourses[n][1]):
        reward = -1
        info['notResource'] += 1

    if sum(action[12:24]) > 100:
      reward = -1
      info['notResource'] += 1

    total_rate = 0.0
    rate, rate_state = self.link_rate(action)

    mMTC_process = 0.0
    for f in range(3):
      mMTC_process += self.VNF_resource[self.mMTC[0][f] - 1][0] * traffic_req[0] * self.mMTC[2] / self.PNs_resourses[0][1]
    mMTC_transmit = 0.0
    mMTC_propag = 0.0
    for index in [0, 1]:
      mMTC_transmit += self.mMTC[2] * traffic_req[0] / rate[int(action[index])][int(action[index + 1])]
      if rate[int(action[index])][int(action[index + 1])] != math.inf:
        total_rate += rate[int(action[index])][int(action[index + 1])]
      distance = math.sqrt(((self.PNs_loc[int(action[index])][0] - self.PNs_loc[int(action[index + 1])][0]) ** 2) + \
      ((self.PNs_loc[int(action[index])][1] - self.PNs_loc[int(action[index + 1])][1]) ** 2))
      mMTC_propag += distance / 200000000.0

    delay[0] = mMTC_transmit + mMTC_process + mMTC_propag
    if delay[0] > self.mMTC[1]:
      reward = -1
      info['notSLA'] += 1

    eMBB_process = 0.0
    for f in range(4):
      eMBB_process += self.VNF_resource[self.eMBB[0][f] - 1][0] * traffic_req[1] * self.eMBB[2] / self.PNs_resourses[0][1]
    eMBB_transmit = 0.0
    eMBB_propag = 0.0
    for index in [3, 4, 5]:
      eMBB_transmit += self.eMBB[2] * traffic_req[1] / rate[int(action[index])][int(action[index + 1])]
      if rate[int(action[index])][int(action[index + 1])] != math.inf:
        total_rate += rate[int(action[index])][int(action[index + 1])]
      distance = math.sqrt(((self.PNs_loc[int(action[index])][0] - self.PNs_loc[int(action[index + 1])][0]) ** 2) + \
      ((self.PNs_loc[int(action[index])][1] - self.PNs_loc[int(action[index + 1])][1]) ** 2))
      eMBB_propag += distance / 200000000.0
    delay[1] = eMBB_transmit + eMBB_process + eMBB_propag
    if delay[1] > self.eMBB[1]:
      reward = -1
      info['notSLA'] += 1

    uRLLC_process = 0.0
    for f in range(5):
      uRLLC_process += self.VNF_resource[self.uRLLC[0][f] - 1][0] * traffic_req[2] * self.uRLLC[2] / self.PNs_resourses[0][1]
    uRLLC_transmit = 0.0
    uRLLC_propag = 0.0
    for index in [7, 8, 9, 10]:
      uRLLC_transmit += self.uRLLC[2] * traffic_req[2] / rate[int(action[index])][int(action[index + 1])]
      if rate[int(action[index])][int(action[index + 1])] != math.inf:
        total_rate += rate[int(action[index])][int(action[index + 1])]
      distance = math.sqrt(((self.PNs_loc[int(action[index])][0] - self.PNs_loc[int(action[index + 1])][0]) ** 2) + \
      ((self.PNs_loc[int(action[index])][1] - self.PNs_loc[int(action[index + 1])][1]) ** 2))
      uRLLC_propag += distance / 200000000.0
    delay[2] = uRLLC_transmit + uRLLC_process + uRLLC_propag
    if delay[2] > self.uRLLC[1]:
      reward = -1
      info['notSLA'] += 1

    cost = [0, 0, 0]
    for s in range(3):
      cost[s] = (traffic_req[s] * 100) #+ ((s + 2) * 1000) + (cpu[s] * 10) + (memory[s] * 10)

    if cost[0] > 100000 or cost[1] > 100000 or cost[2] > 100000:
      reward = -1
      info['notSLA'] += 1

    cpu_state = 12 * [0]
    for index in range(12):
      cpu_state[index] = consumed_cpu[int(action[index])]

    if reward != -1:
      reward = (total_rate / sum(cpu)) / 200000

    # print(action, reward)

    self.state = np.random.choice(range(1, 10), 3)
    self.state = np.concatenate((self.state, rate_state), axis=None)
    self.state = np.concatenate((self.state, cpu_state), axis=None)
    self.state = np.concatenate((self.state, delay), axis=None)

    self.episode_length -= 1
    done = self.episode_length <= 0
    info['delay'] = sum(delay) / 12

    # print(sum(delay))

    # file.write(str(reward) + '\n')

    # Return step information
    return self.state, reward, done, info

  def render(self):
      # Implement viz
      pass

  def reset(self):
      return self.state

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env=myEnv()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


# 5. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
model = DDPG("MlpPolicy", env, verbose=1)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=200000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
|    actor_loss      | -0.0688  |
|    critic_loss     | 0.00118  |
|    learning_rate   | 0.001    |
|    n_updates       | 198666   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0455   |
| time/              |          |
|    episodes        | 98672    |
|    fps             | 33       |
|    time_elapsed    | 6009     |
|    total timesteps | 198671   |
| train/             |          |
|    actor_loss      | -0.0697  |
|    critic_loss     | 0.00136  |
|    learning_rate   | 0.001    |
|    n_updates       | 198670   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0463   |
| time/              |          |
|    episodes        | 98676    |
|    fps             | 33       |
|    time_elapsed    | 6009    

# 6. Save Model

In [ ]:
# model.save('stochastic (50000000000, 50000000000)')

In [ ]:
obs = [2, 3, 1] + 12 * [0] + 12 * [0] + 3 * [0]
avg = 0.0
itr = 0

for req in [1, 3, 5, 7]:
  with open('stochastic (requests=' + str(req) + ').txt', 'a') as file:
    for _ in range(10000):
      obs[0] = np.random.choice(range(req, 10)) + random.uniform(-1, 1)
      obs[1] = np.random.choice(range(req, 10)) + random.uniform(-1, 1)
      obs[2] = np.random.choice(range(req, 10)) + random.uniform(-1, 1)
      action, _states = model.predict(obs)
      obs, reward, dones, info = env.step(action)

      if reward != -1:
        avg = ((avg * itr) + (reward * 200000)) / (itr + 1)
        itr += 1
        file.write(str(avg) + '\n')

In [ ]:
obs = [2, 3, 1] + 12 * [0] + 12 * [0] + 3 * [0]
avg = 0.0
itr = 0

for delay in [1, 3, 5, 7]:
  with open('stochastic (delay=' + str(delay/1000.000) +').txt', 'a') as file:
    for _ in range(10000):
      obs[0] += random.uniform(-1, 1)
      obs[1] += random.uniform(-1, 1)
      obs[2] += random.uniform(-1, 1)

      obs[27] = delay / 1000.000
      obs[28] = delay / 1000.000
      obs[29] = delay / 1000.000

      action, _states = model.predict(obs)
      obs, reward, dones, info = env.step(action)

      if reward != -1:
        avg = ((avg * itr) + (reward * 200000)) / (itr + 1)
        itr += 1
        file.write(str(avg) + '\n')

In [ ]:
obs = [2, 3, 1] + 12 * [0] + 12 * [0] + 3 * [0]
avg = 0.0
itr = 0

for uncertain in [1, 3, 5, 7]:
  with open('stochastic (uncertain=' + str(uncertain) + ').txt', 'a') as file:
    for _ in range(10000):
      obs[0] += random.uniform(-1 * uncertain, uncertain)
      obs[1] += random.uniform(-1 * uncertain, uncertain)
      obs[2] += random.uniform(-1 * uncertain, uncertain)
      action, _states = model.predict(obs)
      obs, reward, dones, info = env.step(action)

      if reward != -1:
        avg = ((avg * itr) + (reward * 200000)) / (itr + 1)
        itr += 1
        file.write(str(avg) + '\n')